# Perform Go Analysis on CellPhoneDB Results

In [ ]:
import numpy as np
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from gprofiler import gprofiler
import gc

## Load Data

In [ ]:
# Set Paths
directory = 'results/method2_withScore'
file_path = directory + '/statistical_analysis_significant_means'

In [ ]:
# Load the interaction data
interaction_data = pd.read_csv(file_path, sep='\t')

# Create a dataframe with cell type pairs and interaction scores
interaction_scores = interaction_data.set_index('interacting_pair').iloc[:, 13:]


## Perform GO Analysis

First, we will take all significant ligand and receptors found from cellphoneDB and perform gene ontology.

This analysis will help us see which pathways our gene sets are statistically enriched

In [ ]:
#Make List objects for ligand and receptors in dataset
interactions = interaction_scores.index.values.tolist()
ligands = [x.split('_')[0] for x in interactions]
receptors = [x.split('_')[1] for x in interactions]

#Perform GO Analysis with gprofiler
ligand_bulk = list(set(ligands))
enrich_bulk = gprofiler(organism='hsapiens',query=ligand_bulk)

#write to file
enrich_bulk.to_csv(os.path.join(directory,'bulk_GO_analysis.csv'))

Next, we investigate significant ligand receptor results by each cell type within our dataset. This gives us an idea of potential differences in signalling and pathway abundance by specific cells

In [ ]:
# Create a dictionary to hold ligands for each cell type
cell_type_ligands = {}

cell_types = interaction_data.columns[14:]
interaction_scores['ligand'] = ligands
interaction_scores['receptor'] = receptors

for cell_type in cell_types:
    # Extract ligands for the cell type
    cell_specific_ligands = interaction_scores[interaction_scores[cell_type] > 0]['ligand'].unique().tolist()
    cell_type_ligands[cell_type] = cell_specific_ligands

# Create a dictionary to hold enrichment results for each cell type
enrichment_results_by_cell_type = {}

for cell_type, ligands in cell_type_ligands.items():
    if ligands:  # Ensure there are ligands to analyze
        enrichment_results = gprofiler(organism='hsapiens', query=ligands)
        if enrichment_results is None:
            enrichment_results_by_cell_type[cell_type] = None
        else:
            significant_results = enrichment_results[enrichment_results['p.value'] < 0.05]
            enrichment_results_by_cell_type[cell_type] = significant_results

In [ ]:
#Write Enrichment Results to Excel File
output_file = os.path.join(directory,'GO_enrichment_results_by_cell_type.xlsx')
if enrichment_results_by_cell_type is not None:
    with pd.ExcelWriter(output_file) as writer:
        for cell_type, results in enrichment_results_by_cell_type.items():
            if results is not None:
                results.to_excel(writer, sheet_name=cell_type)

## Plot Results from GO Analysis

In [ ]:
#Subset to top 20 terms
sig_bulk = enrich_bulk[enrich_bulk['p.value'] < 0.05]
top_results = sig_bulk.sort_values(by='p.value').head(20)
top_results['-log10(p.value)'] = -np.log10(top_results['p.value'])

#Plot Go Terms
plt.figure(figsize=(12, 8))
sns.barplot(x='-log10(p.value)', y='term.name', data=top_results, palette='viridis')
plt.title('Top 20 Enriched GO Terms')
plt.xlabel('-log10(p-value)')
plt.ylabel('GO Term')
plt.tight_layout()
plt.show()
